The next major category in this series is **Business Intelligence**, beginning with **Sales Analytics**.

Sales Analytics is a crucial business intelligence component that focuses on understanding sales performance, identifying trends, analyzing customer behavior, and optimizing sales strategies through data-driven insights.

Based on [`piplist.txt`](./README.md) output, you should have a strong set of libraries available: `pandas` for data manipulation, `numpy` for calculations, `statsmodels` for potential time series analysis, `scikit-learn` for customer segmentation, `mlxtend` for product affinity analysis, and `seaborn`/`matplotlib`/`plotly` for visualization.

**Step 1: Generate Sample Sales Data**

We'll create dummy transactional sales data, including dates, products, categories, quantities, prices, and customer information.

In a new Excel cell, enter `=PY` and paste the following code, then press **Ctrl+Enter**:

In [ ]:
# Generate dummy sales data
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import date, timedelta

fake = Faker()

num_customers = 200
num_products = 50
num_transactions = 5000
start_date = date(2023, 1, 1)
end_date = date(2024, 3, 31)

# Generate dummy products and categories
products = {f'Prod_{i}': random.choice(['Electronics', 'Clothing', 'Home Goods', 'Groceries', 'Books']) for i in range(num_products)}
product_list = list(products.keys())
categories = list(set(products.values()))

# Generate dummy customers
customer_ids = [f'Cust_{i}' for i in range(num_customers)]

data = []

for i in range(num_transactions):
    order_id = f'Order_{i}'
    # Distribute dates somewhat unevenly to simulate real sales patterns
    date_offset = random.randint(0, (end_date - start_date).days)
    transaction_date = start_date + timedelta(days=date_offset)

    customer_id = random.choice(customer_ids)

    # Each order can have multiple items (simplified by adding items directly)
    num_items_in_order = random.randint(1, 5)
    items_in_order = random.sample(product_list, min(num_items_in_order, num_products)) # Pick unique products

    for item_id in items_in_order:
        product_category = products[item_id]
        quantity = random.randint(1, 10)
        price = round(random.uniform(5, 500), 2) # Simulate varied prices
        total_sales = quantity * price

        data.append([order_id, transaction_date, customer_id, item_id, product_category, quantity, price, total_sales])

df_sales = pd.DataFrame(data, columns=['OrderID', 'Date', 'CustomerID', 'ProductID', 'ProductCategory', 'Quantity', 'Price', 'Total_Sales'])

# Ensure Date column is datetime
df_sales['Date'] = pd.to_datetime(df_sales['Date'])

df_sales # Output the DataFrame

**Explanation:**

*   This code generates a DataFrame `df_sales` simulating individual sales line items.
*   It includes columns for Order ID, Date, Customer ID, Product ID, Category, Quantity, Price, and calculated Total Sales.
*   `Faker` is used for generating fake company names (though not used in the final df, but good practice).
*   `random` and `numpy` are used to generate varying quantities, prices, and select random customers and products.
*   The result, `df_sales`, will be spilled into your Excel sheet. Let's assume this data is placed in a range or Table named `SalesData`.

**Step 2: Analyze and Visualize Sales Data**

Now, let's perform some common sales analyses: calculate total sales over time, analyze sales by category, perform a basic customer segmentation based on total spending, and calculate product affinity rules.

In a **new** Excel cell, enter `=PY` and paste the following code. Replace `"SalesData"` with the actual name of the Excel range/Table where your dummy data is. Press **Ctrl+Enter**.

In [ ]:
# Analyze and visualize sales data, perform segmentation and affinity analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans # For customer segmentation
from mlxtend.frequent_patterns import apriori, association_rules # For product affinity

# Load the sales data from Excel
# IMPORTANT: Replace "SalesData" with the actual name of your Excel range or Table
# df_sales = xl("SalesData[#All]", headers=True)

# Ensure 'Date' is a datetime column and 'CustomerID', 'OrderID', 'ProductID' are strings
df_sales['Date'] = pd.to_datetime(df_sales['Date'])
df_sales['CustomerID'] = df_sales['CustomerID'].astype(str)
df_sales['OrderID'] = df_sales['OrderID'].astype(str)
df_sales['ProductID'] = df_sales['ProductID'].astype(str)


# --- Sales Trend Analysis ---
# Aggregate sales by day
daily_sales = df_sales.groupby(df_sales['Date'].dt.date)['Total_Sales'].sum().reset_index()
daily_sales['Date'] = pd.to_datetime(daily_sales['Date']) # Convert date index back to datetime column

# Aggregate sales by month for smoother trend
monthly_sales = df_sales.set_index('Date').resample('ME')['Total_Sales'].sum().reset_index()


# --- Sales by Category Analysis ---
category_sales = df_sales.groupby('ProductCategory')['Total_Sales'].sum().reset_index().sort_values('Total_Sales', ascending=False)


# --- Customer Segmentation (by Total Spending) ---
# Calculate total spending per customer
customer_spending = df_sales.groupby('CustomerID')['Total_Sales'].sum().reset_index()
customer_spending = customer_spending.rename(columns={'Total_Sales': 'Total_Spending'})

# Prepare data for clustering (need a 2D array)
X_spending = customer_spending[['Total_Spending']].values

# Determine optimal number of clusters (e.g., using Elbow method - requires plotting)
# For simplicity here, we'll just pick a number of clusters (e.g., 3)
n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10) # n_init to suppress warning
customer_spending['Cluster'] = kmeans.fit_predict(X_spending)


# --- Product Affinity Analysis (Market Basket Analysis) ---
# Prepare transactional data: list of items per OrderID
# Pivot data to one-hot encode products per order
basket = (df_sales.groupby(['OrderID', 'ProductID'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('OrderID'))

# Convert quantities to binary (1 if purchased, 0 otherwise)
def encode_units(x):
    return 1 if x > 0 else 0

basket_encoded = basket.map(encode_units)

# Find frequent itemsets (combinations of products bought together)
# Use a reasonable minimum support threshold (e.g., 1%) - needs adjustment based on data size
frequent_itemsets = apriori(basket_encoded, min_support=0.01, use_colnames=True) # Adjust min_support as needed

# Generate association rules (e.g., if X is bought, Y is also bought)
# Filter by confidence (e.g., 50%)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5) # Adjust min_threshold as needed

# Sort rules by lift (how much more likely Y is bought when X is bought, compared to random)
rules = rules.sort_values(by='lift', ascending=False)


# --- Visualization ---

# Apply custom style guidelines
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 11
plt.rcParams['axes.edgecolor'] = '#1a1a24'
plt.rcParams['axes.linewidth'] = 1
plt.rcParams['axes.grid'] = False # Turn off default grid
sns.set_theme(style="whitegrid") # Use a seaborn theme base, then apply customs

# 1. Monthly Sales Trend Plot
fig1, ax1 = plt.subplots(figsize=(10, 6))
ax1.plot(monthly_sales['Date'], monthly_sales['Total_Sales'], marker='o', linestyle='-', color='#188ce5') # Blue

ax1.set_title('Monthly Total Sales Trend', fontsize=14, color='#1a1a24')
ax1.set_xlabel('Date', fontsize=12, color='#1a1a24')
ax1.set_ylabel('Total Sales', fontsize=12, color='#1a1a24')
sns.despine(ax=ax1, top=True, right=True)
ax1.grid(False)
fig1.autofmt_xdate()
plt.tight_layout()


# 2. Sales by Category Bar Chart
fig2, ax2 = plt.subplots(figsize=(10, 6))
# Using a categorical color palette from the guidelines
colors = ['#ffe600', '#188ce5', '#2db757', '#ff6d00', '#750e5c'] # Yellow, Blue, Green, Orange, Purple
sns.barplot(x='Total_Sales', y='ProductCategory', hue='ProductCategory', legend=False, data=category_sales, ax=ax2, palette=colors[:len(category_sales)])

ax2.set_title('Total Sales by Product Category', fontsize=14, color='#1a1a24')
ax2.set_xlabel('Total Sales', fontsize=12, color='#1a1a24')
ax2.set_ylabel('Category', fontsize=12, color='#1a1a24')
sns.despine(ax=ax2, top=True, right=True)
ax2.grid(False)

# Add data labels (sales values)
for index, row in category_sales.iterrows():
    ax2.text(row['Total_Sales'], index, f' {row["Total_Sales"]:,.0f}', color='#1a1a24', va='center')

plt.tight_layout()


# 3. Customer Spending Segmentation Plot (Scatter Plot)
# Plotting Total Spending vs Cluster - requires another numerical dimension, or just plot spending and color by cluster
# Simple plot: spending on x-axis, use strip plot or similar, colored by cluster
fig3, ax3 = plt.subplots(figsize=(10, 4)) # Wider for horizontal spread
# Using swarmplot can sometimes show distribution better than scatter for 1D data with categories
sns.swarmplot(x='Total_Spending', y='Cluster', hue='Cluster', legend=False, data=customer_spending, ax=ax3, palette='viridis') # Viridis is a common colormap

ax3.set_title('Customer Segmentation by Total Spending', fontsize=14, color='#1a1a24')
ax3.set_xlabel('Total Spending', fontsize=12, color='#1a1a24')
ax3.set_ylabel('Cluster', fontsize=12, color='#1a1a24')
ax3.grid(False)
sns.despine(ax=ax3, top=True, right=True)
plt.tight_layout()


# Output results
output = {
    'Monthly Sales Trend Head': monthly_sales.head(),
    'Category Sales': category_sales,
    f'Customer Spending (with {n_clusters} Clusters) Head': customer_spending.head(),
    'Association Rules Head (Sorted by Lift)': rules.head(10), # Show top 10 rules
    'Monthly_Sales_Trend_Plot': fig1,
    'Category_Sales_Bar_Chart': fig2,
    'Customer_Segmentation_Plot': fig3,
}

output # Output the dictionary

**Explanation:**

*   We load the dummy sales data. **Remember to replace `"SalesData"`**.
*   We aggregate sales data to calculate total daily and monthly sales using `groupby()` and `resample()`.
*   We calculate total sales for each product category.
*   We perform a basic customer segmentation using K-Means clustering based on the total amount each customer spent.
*   We prepare the data for Market Basket Analysis by transforming it into a one-hot encoded format where each row is an order and columns represent whether a product was in that order.
*   We use `mlxtend`'s `apriori` algorithm to find frequent itemsets (combinations of products bought together often) and `association_rules` to derive rules like "customers who bought X also bought Y". We filter and sort the rules by 'confidence' and 'lift'.
*   **Visualization:**
    *   `fig1`: A line plot showing the monthly total sales trend over time.
    *   `fig2`: A bar chart showing the total sales contributed by each product category.
    *   `fig3`: A plot showing the distribution of customer spending, colored by their assigned cluster from the K-Means analysis. Using `swarmplot` helps visualize individual points for a small number of clusters.
*   **Custom Style:** Applied the specified style guidelines (font, colors, axes, spines, grid). Data labels are added to the bar chart.
*   We return a dictionary containing heads of calculated DataFrames/Series (including the top 10 association rules), and the three plot figures.

**Viewing the Output:**

*   Click the Python cell, then click the Python icon/button next to the formula bar.
*   Select "Excel Value" (**Ctrl+Shift+Alt+M**) for the DataFrames/Series ('Monthly Sales Trend Head', 'Category Sales', 'Customer Spending (with 3 Clusters) Head', 'Association Rules Head (Sorted by Lift)') to spill them into your sheet.
*   For each plot figure object ('Monthly_Sales_Trend_Plot', 'Category_Sales_Bar_Chart', 'Customer_Segmentation_Plot'), select "Picture in Cell" > "Create Reference" to see the plots.

**Further Analysis:**

Here are some advanced sales analytics techniques you could explore:

1. **Advanced Sales Forecasting:**
   - Implement time series decomposition
   - Add seasonal adjustment techniques
   - Create hierarchical forecasting models

2. **Customer Behavior Analysis:**
   - Add cohort analysis
   - Implement customer lifetime value prediction
   - Create churn prediction models

3. **Product Performance Analysis:**
   - Add product cannibalization analysis
   - Implement price elasticity studies
   - Create product lifecycle analysis

4. **Advanced Market Basket Analysis:**
   - Implement sequential pattern mining
   - Add temporal association rules
   - Create recommendation systems

5. **Sales Performance Analytics:**
   - Add sales funnel analysis
   - Implement sales velocity metrics
   - Create win/loss analysis tools

The next topic in the series is [Business Intelligence - Marketing Analysis](./02-Business%20Intelligence_02-Marketing%20Analysis.md), which explores different aspects of marketing performance and campaign effectiveness.